<a href="https://colab.research.google.com/github/agaonsindhe/search-agents/blob/main/ACI_Assignment_1_SolutionTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. ___________________
2. __________________
3. ____________________
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected
IMPORTATANT: Write distinct code block as below

In [ ]:
#Code Block : Set Initial State (Must handle dynamic inputs)
import math
from queue import PriorityQueue
def set_initial_state(start_city=None, end_city=None):
    """
    Sets the initial state for the traveling problem with dynamic inputs.

    :param start_city: The starting city for the travel.
    :param end_city: The destination city for the travel.
    :param kwargs: Additional parameters such as current location's latitude and longitude.
    :return: A dictionary representing the initial state of the problem.
    """
    initial_state = {
        'start_city': start_city,
        'end_city': end_city,
    }

    return initial_state

# Example usage
initial_state = set_initial_state(start_city='Panaji', end_city='Chennai')
print(initial_state)


{'start_city': 'Panaji', 'end_city': 'Chennai'}


In [ ]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)

import math

cities_info = {
    "Panaji": (15.4909,73.8278),
    "Raichur": (16.2076,77.3463),
    "Mangalore": (12.9141,74.8560),
    "Bellari": (15.1394,76.9214),
    "Tirupati": (13.6288, 79.4192),
    "Kurnool": (15.8281,78.0373),
    "Kozhikode": (11.2588,75.7804),
    "Bangalore": (12.9716,77.5946),
    "Nellore": (14.4426,79.9865),
    "Chennai":(13.0827,80.2707)
    # Add other cities here with their lat, long
}


def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def set_transition_and_cost_matrix(cities_info):
    """
    Initializes a matrix/dictionary representing the graph of cities with the cost of travel between them.

    :param cities_info: A dictionary with city names as keys and their lat, long as values.
    :return: A dictionary of dictionaries representing the distances between cities.
    """
    matrix = {}
    for city, (lat1, lon1) in cities_info.items():
        matrix[city] = {}
        for destination, (lat2, lon2) in cities_info.items():
            if city != destination: # Avoid distance to itself
                distance = haversine(lat1, lon1, lat2, lon2)
                matrix[city][destination] = distance
    return matrix

def get_heuristics(goal_city):
    heuristics = {}
    for k in cities_info.keys():
        distance = haversine(*cities_info[k],*cities_info[goal_city])
        heuristics[k]= distance
    return heuristics

transition_cost_matrix = set_transition_and_cost_matrix(cities_info)

import pandas as pd
# Convert to DataFrame for pretty printing
matrix_df = pd.DataFrame(transition_cost_matrix)

# Fill diagonal with 0s for clarity (no distance to itself)
for city in cities_info:
    if city in matrix_df.index:
        matrix_df.loc[city, city] = 0

matrix_df_reordered = matrix_df.reindex(index=cities_info, columns=cities_info)

matrix_df_reordered

# her = get_heuristics("Chennai")

# print(matrix_df_reordered," ",get_heuristics("Chennai"))


,Panaji,Raichur,Mangalore,Bellari,Tirupati,Kurnool,Kozhikode,Bangalore,Nellore,Chennai
Panaji,0.000000,384.703347,307.213146,334.067476,636.349903,452.250193,515.797860,493.225615,671.742877,744.035156
Raichur,384.703347,0.000000,453.789435,127.191089,363.073048,85.058151,575.671548,360.817146,344.493606,468.709079
Mangalore,307.213146,453.789435,0.000000,332.965424,500.196520,471.576519,209.713852,296.849250,579.748933,586.947676
Bellari,334.067476,127.191089,332.965424,0.000000,317.157600,141.997819,448.826986,251.746981,338.511264,427.479325
Tirupati,636.349903,363.073048,500.196520,317.157600,0.000000,286.159311,474.903874,210.532503,109.240956,110.334415
Kurnool,452.250193,85.058151,471.576519,141.997819,286.159311,0.000000,563.591889,321.186102,259.819051,388.599829
Kozhikode,515.797860,575.671548,209.713852,448.826986,474.903874,563.591889,0.000000,274.174956,577.222190,528.508520
Bangalore,493.225615,360.817146,296.849250,251.746981,210.532503,321.186102,274.174956,0.000000,305.804085,290.172025
Nellore,671.742877,344.493606,579.748933,338.511264,109.240956,259.819051,577.222190,305.804085,0.000000,154.297621
Chennai,744.035156,468.709079,586.947676,427.479325,110.334415,388.599829,528.508520,290.172025,154.297621,0.000000


In [ ]:
# Function to reconstruct the path from the start node to the goal node

def reconstruct_path(came_from, start, goal):
    """
    Reconstructs the path from start to goal using the came_from dictionary.

    :param came_from: A dictionary containing the parent of each node
    :param start: The starting city
    :param goal: The goal city
    :return: A list of cities representing the path from the start to the goal
    """
    current = goal
    path = [current]
    while current != start:
        current = came_from[current]
        path.append(current)
    path.reverse()  # reverse the path to get the correct order from start to goal
    return path

# Example usage:
came_from_example = {'Raichur': 'Panaji', 'Kurnool': 'Raichur', 'Chennai': 'Kurnool'}
start_city = 'Panaji'
goal_city = 'Chennai'
reconstructed_path = reconstruct_path(came_from_example, start_city, goal_city)

# Function to display the path beautifully
def display_path_beautifully(path):
    """
    Displays the path in a structured textual representation.

    :param path: A list of cities representing the path from the start to the goal
    """
    for i in range(len(path)):
        if i == len(path) - 1:  # if it's the last city, just print the city
            print(path[i])
        else:  # otherwise, print the city and an arrow
            print(f"{path[i]} -> ", end="")

# Display the path
display_path_beautifully(reconstructed_path)

Panaji -> Raichur -> Kurnool -> Chennai


In [ ]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

# State change function

def transition_model(current_city, graph):
    """
    Returns the successor states from the current city, along with the cost to reach each successor.

    :param current_city: The current city (state).
    :param graph: A dictionary representing the graph of cities, where keys are city names, and values are dictionaries
                  of neighboring cities and the distances to them.
    :return: A list of tuples, where each tuple contains a successor city and the cost to reach it.
    """
    successors = []
    if current_city in graph:
        for neighbor, cost in graph[current_city].items():
            successors.append((neighbor, cost))
    return successors

graph = {
    "Panaji": {"Mangalore": 365 , "Raichur": 457},
    "Mangalore": {"Panaji": 365, "Bangalore": 352, "Kozhikode": 233},
    "Bangalore": {"Mangalore": 352, "Kozhikode": 356, "Bellari": 153, "Chennai": 346},
    "Kozhikode": {"Mangalore": 233, "Bangalore": 356},
    "Tirupati": {"Bellari": 379, "Raichur": 453, "Chennai": 153},
    "Nellore": {"Kurnool": 325, "Chennai": 175},
    "Chennai": {"Tirupati": 153, "Nellore": 175,"Bangalore": 346},
    "Raichur": {"Panaji": 457,"Tirupati": 453,"Kurnool": 100},
    "Bellari": {"Bangalore": 153,"Tirupati": 379},
    "Kurnool": {"Nellore": 325,"Raichur": 100}
}

# Example usage
current_city = 'Panaji'
successors = transition_model(current_city, graph)
print(f"Successors from {current_city}: {successors}")


Successors from Panaji: [('Mangalore', 365), ('Raichur', 457)]


In [ ]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

#goaltest

# Goal Test Function
def is_goal_reached(current_city, goal):
    """
    Test if the current city is the goal.

    :param current_city: The current city
    :param goal: The goal city
    :return: True if the current city is the goal, False otherwise
    """
    return current_city == goal

### 2.	Definition of Algorithm 1 (Greedy BFS)

In [ ]:
#Code Block : Function for algorithm 1 implementation


import heapq

# Modify the Greedy Best First Search Algorithm to include total cost
def greedy_best_first_search(graph, heuristics,initial_state):
    """
    Starts the GBFS algorithm using parameters defined in the initial state.

    :param graph: The graph of the problem.
    :param heuristics: Heuristic values for each city.
    :param initial_state: A dictionary containing the initial state parameters.
    :return: The results of the search.
    """
    start = initial_state['start_city']
    goal = initial_state['end_city']

    # Priority queue for the frontier cities, ordered by heuristic value
    frontier = []
    heapq.heappush(frontier, (heuristics[start], 0, start))  # includes current cost as 0

    # Dictionary to keep track of the path that led to each city
    came_from = {start: None}

    # Dictionary to keep track of the cost to reach each city
    cost_so_far = {start: 0}

    # The set of explored cities to avoid revisiting them
    explored = set()

    # Counters for nodes and edges explored
    nodes_explored = 0
    edges_explored = 0

    # GBFS loop
    while frontier:
        # Choose the city with the lowest heuristic value
        _, current_cost, current_city = heapq.heappop(frontier)

        nodes_explored += 1  # Increment nodes explored when a node is popped from the frontier

        if current_city == goal:
            return reconstruct_path(came_from, start, goal), current_cost, nodes_explored, edges_explored

        # Mark the city as explored
        explored.add(current_city)

        successors = transition_model(current_city, graph)

        # Add neighboring cities to the frontier if they haven't been explored or
        # if a cheaper path to them has been found
        # Use the transition_model function to get successors and their costs
        for neighbor, distance in transition_model(current_city, graph):
            edges_explored += 1  # Increment edges explored for each neighbor examined
            new_cost = current_cost + distance
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                heapq.heappush(frontier, (heuristics[neighbor], new_cost, neighbor))
                came_from[neighbor] = current_city


    # If the goal was never reached, return None
    return None, None,nodes_explored, edges_explored


### 3.	Definition of Algorithm 2 (Genetic)

In [ ]:
#Code Block : Function for algorithm 2 implementation

#genetic

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)

def get_start_end_states(cities):
    """
    Prompt the user to select the start and end states for the journey by number instead of typing names.

    :param cities: A list of available cities.
    :return: A tuple containing the start and end states.
    """
    # Display cities with numbers
    for i, city in enumerate(cities, 1):
        print(f"{i}. {city}")

    while True:
        try:
            start_index = int(input("Select the number for the start city: ")) - 1
            if start_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            end_index = int(input("Select the number for the destination city: ")) - 1
            if end_index not in range(len(cities)):
                print("Invalid selection. Please select a number from the list.")
                continue

            if start_index == end_index:
                print("Start and destination cities cannot be the same. Please select different cities.")
                continue

            start = cities[start_index]
            end = cities[end_index]

            return start, end
        except ValueError:
            print("Invalid input. Please enter a number.")

# Assuming a list of cities based on the given problem
cities_list = ["Panaji", "Raichur", "Mangalore", "Bellari", "Tirupati", "Kurnool", "Kozhikode", "Bangalore", "Nellore", "Chennai"]

# Call the function to get start and end states
# start_city, end_city = get_start_end_states(cities_list)
# print(f"Starting from: {start_city}, Destination: {end_city}")


### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Call the function to get start and end states
start_city, end_city = get_start_end_states(cities_list)

# Prepare initial state
initial_state = set_initial_state(start_city, end_city)

# Implement GBFS with cost
path_gbfs, total_cost_gbfs, node_explored,edge_explored = greedy_best_first_search(graph, get_heuristics(end_city),initial_state)


# Display the path and total cost if a path was found
if path_gbfs:
    display_path_beautifully(path_gbfs)
    print(f"Total cost: {total_cost_gbfs} km")
else:
    print("No path found from Panaji to Chennai using GBFS.")


1. Panaji
2. Raichur
3. Mangalore
4. Bellari
5. Tirupati
6. Kurnool
7. Kozhikode
8. Bangalore
9. Nellore
10. Chennai
Select the number for the start city: 1
Select the number for the destination city: 10
Panaji -> Raichur -> Tirupati -> Chennai
Total cost: 1063 km


In [ ]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))

#genetic returning tuple for outputs

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 1

# time & space of gbfs
import time
start_time = time.perf_counter()
# Implement GBFS with cost
path_gbfs, total_cost_gbfs, node_explored,edge_explored = greedy_best_first_search(graph, get_heuristics(end_city),initial_state)
end_time = time.perf_counter()
print(f"Estimated Time Complexity & Space Complexity for GBFS from {initial_state['start_city']} to {initial_state['end_city']}:")
print(f"Space Complexity : O({node_explored})")
print("Time complexity: %.6f seconds" % (end_time - start_time))



Estimated Time Complexity & Space Complexity for GBFS from Panaji to Chennai:
Space Complexity : O(4)
Time complexity: 0.000099 seconds


In [ ]:
#Code Block : Print the Time & Space complexity of algorithm 2

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : _______________________________________________

________________________________________________________

_________________________________________________________